## Notebook for working with shapely files

### Opened Tuesday 12-13

#### Couldn't create geopanda df correctly when I imported from a csv so the first few cells are all of the transformations from the clean_parse notebook that created a working object

In [1]:
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import os
from area import area

file_path = ('data\kml\My Places_dec_9_22.kml')

with open(file_path) as file:

    xml_data = file.read()

# Initializing soup variables
soup = BeautifulSoup(xml_data, 'xml')

folders = soup.Document.Folder
list = soup.Document.Folder.find_all('Folder')
layers = soup.Document.Folder.Folder
polygons = soup.Document.Folder.Placemark.Polygon

## Create a dataframe to hold the data parsed from xml

df = pd.DataFrame(columns=['field', 'foul', 'fop'])



## Loop through the folders and extract the data


    
i = 0   

for i in range(len(list)):

    folders = list[i]
    field_name = folders.find('name').text
    foul = folders.find_all('coordinates')[0].text
    fop = folders.find_all('coordinates')[1].text

    row = {
        'field': field_name,
        'foul': foul,
        'fop': fop
    }

    i+=1

    df = df.append(row, ignore_index=True)

## Clean up the data

# remove new line and and space characters from coordinates
df = df.replace(r'\n','', regex=True) 
df = df.replace(r'\t','', regex=True) 


## Drop any duplicate rows
df = df.drop_duplicates(subset=['field'], keep='first')

## Drop and rows with empty fields
df = df[(df != 0).all(1)]


## remove numberic characters and . from field names
df['field'] = df['field'].str.replace(r'\d+', '')
    

C:\Users\Justin\AppData\Local\Temp\ipykernel_16088\1978245045.py:48: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(row, ignore_index=True)
C:\Users\Justin\AppData\Local\Temp\ipykernel_16088\1978245045.py:48: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(row, ignore_index=True)
C:\Users\Justin\AppData\Local\Temp\ipykernel_16088\1978245045.py:48: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(row, ignore_index=True)
C:\Users\Justin\AppData\Local\Temp\ipykernel_16088\1978245045.py:48: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(row, ignore_index=True)
C:\Users\Justin\AppData\Loca

In [2]:

## Create a (lat, long) pair for home plate 
## Just grab first uple form the foul ground coordinates

df['home_plate'] = df['foul'].str.split(' ').str[0]

## Calculate the area of each polygon in square meters
df['foul_area'] = df['foul'].apply(lambda x: area({'type': 'Polygon', 'coordinates': [[tuple(map(float, coord.split(','))) for coord in x.split()]]}))
df['fop_area'] = df['fop'].apply(lambda x: area({'type': 'Polygon', 'coordinates': [[tuple(map(float, coord.split(','))) for coord in x.split()]]}))

# Convert the area to square feet
df['foul_ft'] = df['foul_area'].apply(lambda x: x*10.7639)
df['fop_ft'] = df['fop_area'].apply(lambda x: x*10.7639)

# drop the area columns
df = df.drop(['foul_area', 'fop_area'], axis=1)

# calculate the ratio of foul ground to total area
df['foul_pct'] = (df['foul_ft']/(df['fop_ft']+df['foul_ft']))*100

## Convert the coordinates to a list of tuples
df['foul'] = df['foul'].apply(lambda x: [tuple(map(float, coord.split(','))) for coord in x.split()])
df['fop'] = df['fop'].apply(lambda x: [tuple(map(float, coord.split(','))) for coord in x.split()])


## Create shapely polygons from the coordinates

from shapely.geometry import Polygon
import geopandas as gpd

df['foul'] = df['foul'].apply(lambda x: Polygon(x))
df['fop'] = df['fop'].apply(lambda x: Polygon(x))


df.head()


,field,foul,fop,home_plate,foul_ft,fop_ft,foul_pct
0,. Beal City,"POLYGON Z ((-84.9127615 43.6741539 0, -84.9127...","POLYGON Z ((-84.9127615 43.6741539 0, -84.9115...","-84.9127615,43.6741539,0",25698.311886,89488.155852,22.310183
1,. Buchanan,"POLYGON Z ((-86.3709785 41.8239409 0, -86.3717...","POLYGON Z ((-86.3709785 41.8239409 0, -86.3700...","-86.37097850000001,41.8239409,0",22659.935253,88476.136505,20.389361
2,. Grosse Pointe Woods University Liggett,"POLYGON Z ((-82.9007499 42.4260393 0, -82.9002...","POLYGON Z ((-82.9007499 42.4260393 0, -82.8997...","-82.90074989999999,42.4260393,0",22388.473941,83118.818831,21.219835
3,. Jackson Lumen Christi,"POLYGON Z ((-84.4602665 42.2219816 0, -84.4603...","POLYGON Z ((-84.4602665 42.2219816 0, -84.4591...","-84.4602665,42.2219816,0",20497.618836,92031.701897,18.215358
4,. Kalamazoo Christian,"POLYGON Z ((-85.6134503 42.273406 0, -85.61346...","POLYGON Z ((-85.6134503 42.273406 0, -85.61452...","-85.6134503,42.273406,0",23934.310711,92206.406626,20.608027


## end of code from clean_notebook_parse

### returning usualble gdf

In [3]:
df.info()
from shapely.geometry import Point

## Convert the home plate coordinate into a shapely point
df['home_pt'] = df['home_plate'].apply(lambda x: Point(tuple(map(float, x.split(',')))))
df['home_pt'] = df['home_pt'].apply(lambda x: x.buffer(0.0001))


## change the foul and fop polygons strings to linestrings,
df['foul'] = df['foul'].apply(lambda x: x.exterior.coords[:])
df['fop'] = df['fop'].apply(lambda x: x.exterior.coords[:])

## drop the first and last coordinate from the linestrings
df['foul'] = df['foul'].apply(lambda x: x[1:-1])
df['fop'] = df['fop'].apply(lambda x: x[1:-1])



##### WORKING TO HERE

df.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 222 entries, 0 to 246
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   field       222 non-null    object 
 1   foul        222 non-null    object 
 2   fop         222 non-null    object 
 3   home_plate  222 non-null    object 
 4   foul_ft     222 non-null    float64
 5   fop_ft      222 non-null    float64
 6   foul_pct    222 non-null    float64
dtypes: float64(3), object(4)
memory usage: 13.9+ KB


,field,foul,fop,home_plate,foul_ft,fop_ft,foul_pct,home_pt
0,. Beal City,"[(-84.9127521, 43.6750365, 0.0), (-84.9128487,...","[(-84.9115486, 43.6741598, 0.0), (-84.9115566,...","-84.9127615,43.6741539,0",25698.311886,89488.155852,22.310183,"POLYGON ((-84.9126615 43.6741539, -84.91266198..."
1,. Buchanan,"[(-86.3717456, 41.8245526, 0.0), (-86.3711961,...","[(-86.3700572, 41.8245925, 0.0), (-86.3700867,...","-86.37097850000001,41.8239409,0",22659.935253,88476.136505,20.389361,"POLYGON ((-86.3708785 41.8239409, -86.37087898..."
2,. Grosse Pointe Woods University Liggett,"[(-82.9002148, 42.4267599, 0.0), (-82.9002644,...","[(-82.8997751, 42.4256428, 0.0), (-82.8993929,...","-82.90074989999999,42.4260393,0",22388.473941,83118.818831,21.219835,"POLYGON ((-82.90064989999999 42.4260393, -82.9..."
3,. Jackson Lumen Christi,"[(-84.4603248, 42.2228422, 0.0), (-84.4603637,...","[(-84.4591654, 42.2220179, 0.0), (-84.4590689,...","-84.4602665,42.2219816,0",20497.618836,92031.701897,18.215358,"POLYGON ((-84.4601665 42.2219816, -84.46016698..."
4,. Kalamazoo Christian,"[(-85.613464, 42.2725382, 0.0), (-85.6133923, ...","[(-85.6145228, 42.2734125, 0.0), (-85.6146321,...","-85.6134503,42.273406,0",23934.310711,92206.406626,20.608027,"POLYGON ((-85.6133503 42.273406, -85.613350781..."


In [4]:
from shapely.geometry import LineString

## make a line out of the fop coordinates
df['fop_line'] = df['fop'].apply(lambda x: LineString(x))

df.head()


,field,foul,fop,home_plate,foul_ft,fop_ft,foul_pct,home_pt,fop_line
0,. Beal City,"[(-84.9127521, 43.6750365, 0.0), (-84.9128487,...","[(-84.9115486, 43.6741598, 0.0), (-84.9115566,...","-84.9127615,43.6741539,0",25698.311886,89488.155852,22.310183,"POLYGON ((-84.9126615 43.6741539, -84.91266198...","LINESTRING Z (-84.9115486 43.6741598 0, -84.91..."
1,. Buchanan,"[(-86.3717456, 41.8245526, 0.0), (-86.3711961,...","[(-86.3700572, 41.8245925, 0.0), (-86.3700867,...","-86.37097850000001,41.8239409,0",22659.935253,88476.136505,20.389361,"POLYGON ((-86.3708785 41.8239409, -86.37087898...","LINESTRING Z (-86.3700572 41.8245925 0, -86.37..."
2,. Grosse Pointe Woods University Liggett,"[(-82.9002148, 42.4267599, 0.0), (-82.9002644,...","[(-82.8997751, 42.4256428, 0.0), (-82.8993929,...","-82.90074989999999,42.4260393,0",22388.473941,83118.818831,21.219835,"POLYGON ((-82.90064989999999 42.4260393, -82.9...","LINESTRING Z (-82.8997751 42.4256428 0, -82.89..."
3,. Jackson Lumen Christi,"[(-84.4603248, 42.2228422, 0.0), (-84.4603637,...","[(-84.4591654, 42.2220179, 0.0), (-84.4590689,...","-84.4602665,42.2219816,0",20497.618836,92031.701897,18.215358,"POLYGON ((-84.4601665 42.2219816, -84.46016698...","LINESTRING Z (-84.4591654 42.2220179 0, -84.45..."
4,. Kalamazoo Christian,"[(-85.613464, 42.2725382, 0.0), (-85.6133923, ...","[(-85.6145228, 42.2734125, 0.0), (-85.6146321,...","-85.6134503,42.273406,0",23934.310711,92206.406626,20.608027,"POLYGON ((-85.6133503 42.273406, -85.613350781...","LINESTRING Z (-85.6145228 42.2734125 0, -85.61..."


### I think I have a usable linestring to measure to.

#### for some reason home_pt is getting saved as a polygon, need to figure that out

## below I is for trying to measure the distance


In [21]:
#### trying to work with a function i found from stackexchange

# https://gis.stackexchange.com/questions/376696/geopandas-distance-optimisation

import geopandas as gpd
from shapely.geometry import Point
import pandas as pd
import random

gdf = gpd.GeoDataFrame(geometry=df['fop_line'], crs="EPSG:4326")
gdf2 = gpd.GeoDataFrame(geometry=df['home_pt'], crs="EPSG:4326")

def get_max_distance(left, right, initial_buffer):
    """get distance from left to right"""
    buffered = left.buffer(initial_buffer)
    b = initial_buffer

    distances = []
    for i in range(len(buffered)):
        geom = buffered.geometry.iloc[i]
        query = right.sindex.query(geom)
        while query.size == 0:
            query = right.sindex.query(geom.buffer(b))
            b += initial_buffer
        distances.append(right.iloc[query].distance(left.geometry.iloc[i]).max())

    return pd.Series(distances, index=left.index)

gdf['distance_to_x_max'] = get_max_distance(gdf, gdf2, 50)

gdf['dist_in_meters_max'] = gdf['distance_to_x_max'] * 111000
gdf['dist_in_feet_max'] = gdf['dist_in_meters_max'] * 3.28084

df['max_dist'] = gdf['dist_in_feet_max']

df.head()

C:\Users\Justin\AppData\Local\Temp\ipykernel_16088\2082656974.py:15: UserWarning: Geometry is in a geographic CRS. Results from 'buffer' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  buffered = left.buffer(initial_buffer)
C:\Users\Justin\AppData\Local\Temp\ipykernel_16088\2082656974.py:25: UserWarning: Geometry is in a geographic CRS. Results from 'distance' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  distances.append(right.iloc[query].distance(left.geometry.iloc[i]).max())
C:\Users\Justin\AppData\Local\Temp\ipykernel_16088\2082656974.py:25: UserWarning: Geometry is in a geographic CRS. Results from 'distance' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  distances.append(right.iloc[query].distance(left.geometry.iloc[i]).max())
C:\Users\Justin\AppData\Local\Temp\ipykernel_16

,field,foul,fop,home_plate,foul_ft,fop_ft,foul_pct,home_pt,fop_line,max_dist,min_dist
0,. Beal City,"[(-84.9127521, 43.6750365, 0.0), (-84.9128487,...","[(-84.9115486, 43.6741598, 0.0), (-84.9115566,...","-84.9127615,43.6741539,0",25698.311886,89488.155852,22.310183,"POLYGON ((-84.9126615 43.6741539, -84.91266198...","LINESTRING Z (-84.9115486 43.6741598 0, -84.91...",7.451076e+06,284.875132
1,. Buchanan,"[(-86.3717456, 41.8245526, 0.0), (-86.3711961,...","[(-86.3700572, 41.8245925, 0.0), (-86.3700867,...","-86.37097850000001,41.8239409,0",22659.935253,88476.136505,20.389361,"POLYGON ((-86.3708785 41.8239409, -86.37087898...","LINESTRING Z (-86.3700572 41.8245925 0, -86.37...",6.823488e+06,295.064123
2,. Grosse Pointe Woods University Liggett,"[(-82.9002148, 42.4267599, 0.0), (-82.9002644,...","[(-82.8997751, 42.4256428, 0.0), (-82.8993929,...","-82.90074989999999,42.4260393,0",22388.473941,83118.818831,21.219835,"POLYGON ((-82.90064989999999 42.4260393, -82.9...","LINESTRING Z (-82.8997751 42.4256428 0, -82.89...",8.104590e+06,290.490250
3,. Jackson Lumen Christi,"[(-84.4603248, 42.2228422, 0.0), (-84.4603637,...","[(-84.4591654, 42.2220179, 0.0), (-84.4590689,...","-84.4602665,42.2219816,0",20497.618836,92031.701897,18.215358,"POLYGON ((-84.4601665 42.2219816, -84.46016698...","LINESTRING Z (-84.4591654 42.2220179 0, -84.45...",7.531636e+06,277.727685
4,. Kalamazoo Christian,"[(-85.613464, 42.2725382, 0.0), (-85.6133923, ...","[(-85.6145228, 42.2734125, 0.0), (-85.6146321,...","-85.6134503,42.273406,0",23934.310711,92206.406626,20.608027,"POLYGON ((-85.6133503 42.273406, -85.613350781...","LINESTRING Z (-85.6145228 42.2734125 0, -85.61...",7.116789e+06,287.419549


In [15]:


def get_nearest_distance(left, right, initial_buffer):
    """get distance from left to right"""
    buffered = left.buffer(initial_buffer)
    b = initial_buffer

    distances = []
    for i in range(len(buffered)):
        geom = buffered.geometry.iloc[i]
        query = right.sindex.query(geom)
        while query.size == 0:
            query = right.sindex.query(geom.buffer(b))
            b += initial_buffer
        distances.append(right.iloc[query].distance(left.geometry.iloc[i]).min())

    return pd.Series(distances, index=left.index)

gdf['distance_to_x'] = get_nearest_distance(gdf, gdf2, 50)

gdf['dist_in_meters'] = gdf['distance_to_x'] * 111000
gdf['dist_in_feet'] = gdf['dist_in_meters'] * 3.28084

df['min_dist'] = gdf['dist_in_feet']

df.head()

C:\Users\Justin\AppData\Local\Temp\ipykernel_16088\3151819982.py:3: UserWarning: Geometry is in a geographic CRS. Results from 'buffer' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  buffered = left.buffer(initial_buffer)
C:\Users\Justin\AppData\Local\Temp\ipykernel_16088\3151819982.py:13: UserWarning: Geometry is in a geographic CRS. Results from 'distance' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  distances.append(right.iloc[query].distance(left.geometry.iloc[i]).min())
C:\Users\Justin\AppData\Local\Temp\ipykernel_16088\3151819982.py:13: UserWarning: Geometry is in a geographic CRS. Results from 'distance' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  distances.append(right.iloc[query].distance(left.geometry.iloc[i]).min())
C:\Users\Justin\AppData\Local\Temp\ipykernel_160

,field,foul,fop,home_plate,foul_ft,fop_ft,foul_pct,home_pt,fop_line,max_dist,min_dist
0,. Beal City,"[(-84.9127521, 43.6750365, 0.0), (-84.9128487,...","[(-84.9115486, 43.6741598, 0.0), (-84.9115566,...","-84.9127615,43.6741539,0",25698.311886,89488.155852,22.310183,"POLYGON ((-84.9126615 43.6741539, -84.91266198...","LINESTRING Z (-84.9115486 43.6741598 0, -84.91...",7.451076e+06,284.875132
1,. Buchanan,"[(-86.3717456, 41.8245526, 0.0), (-86.3711961,...","[(-86.3700572, 41.8245925, 0.0), (-86.3700867,...","-86.37097850000001,41.8239409,0",22659.935253,88476.136505,20.389361,"POLYGON ((-86.3708785 41.8239409, -86.37087898...","LINESTRING Z (-86.3700572 41.8245925 0, -86.37...",6.823488e+06,295.064123
2,. Grosse Pointe Woods University Liggett,"[(-82.9002148, 42.4267599, 0.0), (-82.9002644,...","[(-82.8997751, 42.4256428, 0.0), (-82.8993929,...","-82.90074989999999,42.4260393,0",22388.473941,83118.818831,21.219835,"POLYGON ((-82.90064989999999 42.4260393, -82.9...","LINESTRING Z (-82.8997751 42.4256428 0, -82.89...",8.104590e+06,290.490250
3,. Jackson Lumen Christi,"[(-84.4603248, 42.2228422, 0.0), (-84.4603637,...","[(-84.4591654, 42.2220179, 0.0), (-84.4590689,...","-84.4602665,42.2219816,0",20497.618836,92031.701897,18.215358,"POLYGON ((-84.4601665 42.2219816, -84.46016698...","LINESTRING Z (-84.4591654 42.2220179 0, -84.45...",7.531636e+06,277.727685
4,. Kalamazoo Christian,"[(-85.613464, 42.2725382, 0.0), (-85.6133923, ...","[(-85.6145228, 42.2734125, 0.0), (-85.6146321,...","-85.6134503,42.273406,0",23934.310711,92206.406626,20.608027,"POLYGON ((-85.6133503 42.273406, -85.613350781...","LINESTRING Z (-85.6145228 42.2734125 0, -85.61...",7.116789e+06,287.419549


In [ ]:


## Export the data to a csv file to check the results

export_df = df.filter(['field', 'foul_ft', 'fop_ft', 'foul_pct', 'dist_calc'], axis=1)
export_df.to_csv('dist_calc.csv', index=False)

#### Output Looks Pretty Good!!!
### values are right around where i would expect them to be



In [ ]:
#### 12-14-.....1 am
### Don't really understand what this method is returning
### average distance maybe?
### avg distance would be good to know but what I am really interested in is the max and min distances



import shapely.wkt
import geopandas as gpd

gdf_line = gpd.GeoDataFrame(geometry=df['fop_line'], crs="EPSG:4326")

gdf_hp = gpd.GeoDataFrame(geometry=df['home_pt'], crs="EPSG:4326")
utm = gdf_line.estimate_utm_crs()

for x in gdf_line:
    gdf_line.to_crs(utm)
    df['avg_dist'] = (gdf_line.to_crs(utm).distance(gdf_hp.to_crs(utm)))*3.28084
    df['max_dist'] = (gdf_line.to_crs(utm).distance(gdf_hp.to_crs(utm))


In [ ]:
df.head()

In [ ]:
from pyproj import Geod

geod = Geod(ellps="WGS84")

f"{geod.geometry_length(df['fop_line'][10])} meters"
f"{geod.geometry_length(df['fop_line'][10])*3.28084} feet"

## calculate the distance from home plate to the fop line
df['dist_to_fop'] = df['home_pt'].apply(lambda x: geod.geo(df['fop_line'][10]))

df.head()


In [ ]:




## Calculate the distance from home plate to the outfield line
df['home_pt'] = Point(df['home_plate'])
df['dist'] = df.apply(lambda x: x['home_pt'].distance(x['fop_line']), axis=1)
df.head()

In [ ]:
### Import folium and create map
x = 40

home_pt = Point(df['home_pt'][x])

foul_poly = df['foul'][x]
fop_poly = df['fop'][x]

foul_geom = Polygon(foul_poly)
fop_geom = Polygon(fop_poly)
polygon = gpd.GeoDataFrame(index=[x], crs='epsg:3857', geometry=[fop_geom])       


print(polygon.boundary.distance(home_pt))

test_pt = polygon.exterior.interpolate(polygon.exterior.project(home_pt))

# print(test_pt)
# print(home_pt)
# print(test_pt.distance(home_pt))

# 1 METER = 3.28084 FEET


# import folium
# m = folium.Map(home, zoom_start=5, tiles='cartodbpositron')
# folium.GeoJson(polygon).add_to(m)
# folium.LatLngPopup().add_to(m)
# m

In [ ]:
x = 12

home = [df['home_lat'][x], df['home_long'][x]]

foul_poly = df['foul'][x]
fop_poly = df['fop'][x]

foul_geom = Polygon(foul_poly)
fop_geom = Polygon(fop_poly)

df['foul'].boundary.distance(home)

In [ ]:
# https://gis.stackexchange.com/questions/294206/how-to-create-a-simple-polygon-from-coordinates-in-geopandas-with-python

# from shapely import wkt

# import geopandas as gpd
# from shapely.geometry import Polygon

# lon_lat_list = df['foul'][3]
# polygon_geom = Polygon(lon_lat_list)
# polygon = gpd.GeoDataFrame(index=[0], crs='epsg:4326', geometry=[polygon_geom])       

# import folium
# m = folium.Map([50.854457, 4.377184], zoom_start=5, tiles='cartodbpositron')
# folium.GeoJson(polygon).add_to(m)
# folium.LatLngPopup().add_to(m)
# m
# df['foul_shapely'] = df['foul_str'].apply(lambda x: wkt.loads(x)) # only works with strings
# df.head()